In [13]:
import mysql.connector
from tkinter import *
from tkinter import messagebox, ttk
from datetime import date

# MySQL Connection
conn = mysql.connector.connect(host="localhost", user="root", password="root", database="pharmacy1")
cursor = conn.cursor()

root = Tk()
root.title("Pharmacy Management System")

def main_dashboard():
    for widget in root.winfo_children():
        widget.destroy()

    Label(root, text="Welcome to Pharmacy Management System", font=("Helvetica", 16)).grid(row=0, columnspan=2)
    Button(root, text="View All Medicines", width=20, command=display_medicines).grid(row=1, column=0, padx=10, pady=10)
    Button(root, text="Manage Medicines", width=20, command=manage_medicines).grid(row=1, column=1, padx=10, pady=10)
    Button(root, text="Billing", width=20, command=billing).grid(row=2, column=0, padx=10, pady=10)
    Button(root, text="View All Bills", width=20, command=view_bills).grid(row=2, column=1, padx=10, pady=10)
    Button(root, text="View All Customers", width=20, command=view_customers).grid(row=3, column=0, columnspan=2, pady=10)
    Button(root, text="Filters & Reports", width=20, command=filters).grid(row=4, column=0, columnspan=2, pady=10)

def display_medicines():
    for widget in root.winfo_children():
        widget.destroy()
    cursor.execute("SELECT id, name, batch_no, expiry_date, price, quantity FROM medicines")
    medicines = cursor.fetchall()
    Label(root, text="All Medicines", font=("Helvetica", 16)).grid(row=0, columnspan=6)
    tree = ttk.Treeview(root, columns=("ID", "Name", "Batch No", "Expiry Date", "Price", "Quantity"), show="headings")
    for col in tree["columns"]:
        tree.heading(col, text=col)
    for med in medicines:
        tree.insert("", "end", values=med)
    tree.grid(row=1, columnspan=6, padx=10, pady=10)
    Button(root, text="Back to Dashboard", width=20, command=main_dashboard).grid(row=2, columnspan=6)

def manage_medicines():
    for widget in root.winfo_children():
        widget.destroy()
    Label(root, text="Manage Medicines", font=("Helvetica", 16)).grid(row=0, columnspan=2)
    Button(root, text="Add Medicine", width=20, command=add_medicine).grid(row=1, column=0, padx=10, pady=10)
    Button(root, text="Update Medicine", width=20, command=update_medicine).grid(row=1, column=1, padx=10, pady=10)
    Button(root, text="Back to Dashboard", width=20, command=main_dashboard).grid(row=2, columnspan=2)

def add_medicine():
    for widget in root.winfo_children():
        widget.destroy()
    Label(root, text="Enter Medicine Details", font=("Helvetica", 16)).grid(row=0, columnspan=2)
    entries = {}
    fields = ["Name", "Batch No", "Expiry Date", "Price", "Quantity"]
    for i, field in enumerate(fields, start=1):
        Label(root, text=field + ":").grid(row=i, column=0)
        entries[field] = Entry(root)
        entries[field].grid(row=i, column=1)

    def save():
        values = [entries[field].get() for field in fields]
        cursor.execute("INSERT INTO medicines (name, batch_no, expiry_date, price, quantity) VALUES (%s, %s, %s, %s, %s)", values)
        conn.commit()
        messagebox.showinfo("Success", "Medicine Added Successfully!")
        manage_medicines()

    Button(root, text="Save Medicine", command=save).grid(row=6, columnspan=2, pady=10)
    Button(root, text="Back to Manage Medicines", width=20, command=manage_medicines).grid(row=7, columnspan=2)

def update_medicine():
    for widget in root.winfo_children():
        widget.destroy()
    Label(root, text="Enter Medicine ID to Update", font=("Helvetica", 16)).grid(row=0, columnspan=2)
    Label(root, text="Medicine ID:").grid(row=1, column=0)
    entry_id = Entry(root)
    entry_id.grid(row=1, column=1)
    Label(root, text="New Quantity:").grid(row=2, column=0)
    entry_quantity = Entry(root)
    entry_quantity.grid(row=2, column=1)

    def save_updated():
        cursor.execute("UPDATE medicines SET quantity=%s WHERE id=%s", (entry_quantity.get(), entry_id.get()))
        conn.commit()
        messagebox.showinfo("Success", "Medicine Updated Successfully!")
        manage_medicines()

    Button(root, text="Update Medicine", command=save_updated).grid(row=3, columnspan=2, pady=10)
    Button(root, text="Back to Manage Medicines", width=20, command=manage_medicines).grid(row=4, columnspan=2)

def billing():
    for widget in root.winfo_children():
        widget.destroy()
    Label(root, text="Billing", font=("Helvetica", 16)).grid(row=0, columnspan=2)

    Label(root, text="Customer ID:").grid(row=1, column=0)
    entry_customer_id = Entry(root)
    entry_customer_id.grid(row=1, column=1)

    Label(root, text="Gender (M/F):").grid(row=2, column=0)
    entry_gender = Entry(root)
    entry_gender.grid(row=2, column=1)

    Label(root, text="Medicine ID:").grid(row=3, column=0)
    entry_med_id = Entry(root)
    entry_med_id.grid(row=3, column=1)

    Label(root, text="Quantity:").grid(row=4, column=0)
    entry_quantity = Entry(root)
    entry_quantity.grid(row=4, column=1)

    tree = ttk.Treeview(root, columns=("Medicine", "Quantity", "Price", "Total"), show="headings")
    for col in tree["columns"]:
        tree.heading(col, text=col)
    tree.grid(row=5, columnspan=2, pady=10)

    items = []
    total_amount = [0]

    def add_to_bill():
        med_id = entry_med_id.get()
        quantity = int(entry_quantity.get())
        cursor.execute("SELECT name, price, quantity FROM medicines WHERE id = %s", (med_id,))
        medicine = cursor.fetchone()
        if medicine:
            name, price, stock = medicine
            if quantity <= stock:
                total = price * quantity
                tree.insert("", "end", values=(name, quantity, price, total))
                total_amount[0] += total
                items.append(f"{name} x{quantity}")
                cursor.execute("UPDATE medicines SET quantity = %s WHERE id = %s", (stock - quantity, med_id))
                conn.commit()
                entry_med_id.delete(0, END)
                entry_quantity.delete(0, END)
            else:
                messagebox.showerror("Error", "Insufficient stock.")
        else:
            messagebox.showerror("Error", "Medicine not found.")

    def finalize_bill():
        customer_id = entry_customer_id.get()
        gender = entry_gender.get().upper()
        bill_text = ", ".join(items)
        bill_total = total_amount[0]
        bill_date = date.today()

        # Check if customer exists, if not insert
        cursor.execute("SELECT customer_id FROM customer WHERE customer_id = %s", (customer_id,))
        if not cursor.fetchone():
            cursor.execute("INSERT INTO customer (customer_id, gender) VALUES (%s, %s)", (customer_id, gender))
            conn.commit()

        cursor.execute("INSERT INTO bill (bill_amount, bill_date, bill_items, customer_id) VALUES (%s, %s, %s, %s)",
                       (bill_total, bill_date, bill_text, customer_id))
        cursor.execute("UPDATE customer SET customer_bills = customer_bills + 1, total_order_amount = total_order_amount + %s WHERE customer_id = %s",
                       (bill_total, customer_id))
        conn.commit()
        messagebox.showinfo("Success", f"Bill of ₹{bill_total:.2f} generated for customer ID {customer_id}.")
        main_dashboard()

    Button(root, text="Add to Bill", command=add_to_bill).grid(row=6, column=0, pady=10)
    Button(root, text="Finalize Bill", command=finalize_bill).grid(row=6, column=1, pady=10)
    Button(root, text="Back to Dashboard", command=main_dashboard).grid(row=7, columnspan=2, pady=10)

def view_bills():
    for widget in root.winfo_children():
        widget.destroy()
    Label(root, text="All Bills", font=("Helvetica", 16)).grid(row=0, columnspan=5)
    cursor.execute("SELECT * FROM bill")
    bills = cursor.fetchall()
    tree = ttk.Treeview(root, columns=("ID", "Amount", "Date", "Items", "Customer ID"), show="headings")
    for col in tree["columns"]:
        tree.heading(col, text=col)
    for bill in bills:
        tree.insert("", "end", values=bill)
    tree.grid(row=1, columnspan=5, padx=10, pady=10)
    Button(root, text="Back to Dashboard", width=20, command=main_dashboard).grid(row=2, columnspan=5)

def view_customers():
    for widget in root.winfo_children():
        widget.destroy()
    Label(root, text="All Customers", font=("Helvetica", 16)).grid(row=0, columnspan=4)
    cursor.execute("SELECT * FROM customer")
    customers = cursor.fetchall()
    tree = ttk.Treeview(root, columns=("ID", "Gender", "Bills", "Total Amount"), show="headings")
    for col in tree["columns"]:
        tree.heading(col, text=col)
    for cust in customers:
        tree.insert("", "end", values=cust)
    tree.grid(row=1, columnspan=4, padx=10, pady=10)
    Button(root, text="Back to Dashboard", width=20, command=main_dashboard).grid(row=2, columnspan=4)

def filters():
    for widget in root.winfo_children():
        widget.destroy()

    Label(root, text="Filters & Reports", font=("Helvetica", 16)).grid(row=0, columnspan=2, pady=10)

    def show_expired():
        date_str = entry_expiry.get()
        cursor.execute("""
            SELECT * FROM medicines 
            WHERE expiry_date < %s
        """, (date_str,))
        show_results(cursor.fetchall(), ("ID", "Name", "Batch", "Expiry", "Price", "Quantity"))

    def show_sales_by_date():
        date_str = entry_sales_date.get()
        cursor.execute("""
            SELECT IFNULL(SUM(bill_amount), 0) 
            FROM bill 
            WHERE bill_date = %s
        """, (date_str,))
        result = cursor.fetchone()
        messagebox.showinfo("Total Sales", f"Total sales on {date_str}: ₹{result[0]:.2f}")

    def show_top_selling():
        cursor.execute("""
            SELECT name, 
                   (SELECT SUM(CAST(SUBSTRING_INDEX(SUBSTRING_INDEX(bill_items, CONCAT(name, ' x'), -1), ',', 1) AS UNSIGNED))
                    FROM bill 
                    WHERE bill_items LIKE CONCAT('%', name, ' x%')) AS total_sold
            FROM medicines 
            ORDER BY total_sold DESC 
            LIMIT 5
        """)
        show_results(cursor.fetchall(), ("Medicine", "Total Sold"))

    def show_low_stock():
        cursor.execute("""
            SELECT * FROM medicines 
            WHERE quantity < 20
        """)
        show_results(cursor.fetchall(), ("ID", "Name", "Batch", "Expiry", "Price", "Quantity"))

    def show_top_customers():
        cursor.execute("""
            SELECT customer_id, total_order_amount 
            FROM customer 
            WHERE total_order_amount = (
                SELECT MAX(total_order_amount) 
                FROM customer
            )
            LIMIT 5
        """)
        show_results(cursor.fetchall(), ("Customer ID", "Total Purchase"))

    # UI Inputs
    Label(root, text="Expiry before (YYYY-MM-DD):").grid(row=1, column=0, sticky="e")
    entry_expiry = Entry(root)
    entry_expiry.grid(row=1, column=1)
    Button(root, text="Find Expired", command=show_expired).grid(row=1, column=2)

    Label(root, text="Sales on (YYYY-MM-DD):").grid(row=2, column=0, sticky="e")
    entry_sales_date = Entry(root)
    entry_sales_date.grid(row=2, column=1)
    Button(root, text="Total Sales", command=show_sales_by_date).grid(row=2, column=2)

    Button(root, text="Top 5 Best Sellers", command=show_top_selling).grid(row=3, column=0, columnspan=2, pady=5)
    Button(root, text="Low Stock (<20)", command=show_low_stock).grid(row=4, column=0, columnspan=2, pady=5)
    Button(root, text="Top Customers", command=show_top_customers).grid(row=5, column=0, columnspan=2, pady=5)

    Button(root, text="Back to Dashboard", width=20, command=main_dashboard).grid(row=6, column=0, columnspan=3, pady=10)

def show_results(rows, columns):
    result_window = Toplevel(root)
    result_window.title("Results")
    tree = ttk.Treeview(result_window, columns=columns, show="headings")
    for col in columns:
        tree.heading(col, text=col)
    for row in rows:
        tree.insert("", "end", values=row)
    tree.pack(expand=True, fill="both")


main_dashboard()
root.mainloop()
conn.close()